# LSTM Notes

## LSTM Cell
![LSTM](/Users/jerrykim/workspace/deep-learning/tensorflow/lstm-input-gate.png)
## LSTM Gate Operation
![GATE](/Users/jerrykim/workspace/deep-learning/tensorflow/lstm.png)


In [11]:
from tqdm import tqdm
import numpy as np
import re

In [12]:
data = """
우리 나라 말이 중국과 달라 한자와는 서로 말이 통하지 아니하여서 이런 까닭으로 어리석은 백성이 말하고자 할 것이 있어도 마침내 제 뜻을 펴지 못하는 사람이 많다. 내가 이것을 가엾게 여겨 새로 스물 여덟 글자를 만드니 모든 사람들로 하여금 쉽게 익혀서 날마다 쓰는 데 편하게 하고자 할 따름이니라."""

In [13]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def tanh(x, derivative=False):
    return np.tanh(x)

def tanh_derivative(x):
    return 1 - x ** 2

def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))

class MyLSTM(object):
    def __init__(self, source, hidden_size, learning_rate=0.01):
        self.Y = None
        self.I = None
        self.F = None
        self.O = None
        self.C = None
        self.CS = None
        self.HS = None
        self.X = None
        self.idx_to_word = None
        self.word_to_idx = None
        self.vocab = None
        self.tokens = None
        self.source = source
        self.hidden_size = hidden_size        
        self.learning_rate = learning_rate
        self.data_preprocessing(source)
        self.input_size = len(self.vocab) + self.hidden_size # input_size = vocab_size + hidden_size
        self.output_size = len(self.vocab) # output_size = vocab_size
        
        ## weight initialization
        ## forget gate
        self.W_f = np.random.randn(self.hidden_size, self.input_size) * 0.1
        self.b_f = np.zeros((self.hidden_size, 1))
        ## input gate
        self.W_i = np.random.randn(self.hidden_size, self.input_size) * 0.1
        self.b_i = np.zeros((self.hidden_size, 1))
        ## cell gate
        self.W_c = np.random.randn(self.hidden_size, self.input_size) * 0.1
        self.b_c = np.zeros((self.hidden_size, 1))
        ## output gate
        self.W_o = np.random.randn(self.hidden_size, self.input_size) * 0.1
        self.b_o = np.zeros((self.hidden_size, 1))
        ## final output
        self.W_y = np.random.randn(self.output_size, self.hidden_size) 
        self.b_y = np.zeros((self.output_size, 1))
    
    def reset(self):
        self.X = {}
        self.HS = {-1: np.zeros((self.hidden_size, 1))}
        self.CS = {-1: np.zeros((self.hidden_size, 1))}
        self.C = {}
        self.O = {}
        self.F = {}
        self.I = {}
        self.Y = {}
        
    def data_preprocessing(self, source):
        source = re.sub(r'[^가-힣]', '', source)
        self.tokens = data.split()
        self.vocab = list(set(self.tokens))
        self.word_to_idx = {w:i for i, w in enumerate(self.vocab)}
        self.idx_to_word = {i:w for i, w in enumerate(self.vocab)}

    def one_hot_encoding(self, idx):
        size = len(self.vocab)
        one_hot_vector = np.zeros((size, 1))
        one_hot_vector[idx][0] = 1
        return one_hot_vector
    
    def forward(self, inputs):
        x = {}
        outputs = []
        for t in range(len(inputs)):
            x[t] = self.one_hot_encoding(self.word_to_idx[inputs[t]])
            self.X[t] = np.row_stack((x[t], self.HS[t-1]))
            self.F[t] = sigmoid(np.dot(self.W_f, self.X[t]) + self.b_f)
            self.I[t] = sigmoid(np.dot(self.W_i, self.X[t]) + self.b_i)
            self.C[t] = tanh(np.dot(self.W_c, self.X[t]) + self.b_c)
            self.O[t] = sigmoid(np.dot(self.W_o, self.X[t]) + self.b_o)
            self.CS[t] = self.F[t] * self.CS[t-1] + self.I[t] * self.C[t]
            self.HS[t] = self.O[t] * tanh(self.CS[t])
            self.Y[t] = softmax(np.dot(self.W_y, self.HS[t]) + self.b_y)
            outputs += [np.dot(self.W_y, self.HS[t]) + self.b_y]
        
        return outputs
    
    def backward(self, errors, inputs):
        dW_f = np.zeros_like(self.W_f)
        dW_i = np.zeros_like(self.W_i)
        dW_c = np.zeros_like(self.W_c)
        dW_o = np.zeros_like(self.W_o)
        dW_y = np.zeros_like(self.W_y)
        db_f = np.zeros_like(self.b_f)
        db_i = np.zeros_like(self.b_i)
        db_c = np.zeros_like(self.b_c)
        db_o = np.zeros_like(self.b_o)
        db_y = np.zeros_like(self.b_y)
        dHS_next = np.zeros_like(self.HS[0])
        dCS_next = np.zeros_like(self.CS[0])
        
        for t in reversed(range(len(inputs))):
            error = errors[t]
            # final Gate            
            dW_y += np.dot(error, self.HS[t].T)
            db_y += error
            
            # hidden state error
            dHS = np.dot(self.W_y.T, error) + dHS_next
            
            # output gate error
            dO = dHS * tanh(self.CS[t]) * sigmoid_derivative(self.O[t])            
            dW_o += np.dot(dO, self.X[t].T)
            db_o += dO
            
            # cell state error
            dCS = dHS * self.O[t] * tanh_derivative(self.CS[t]) + dCS_next
            
            # input gate error
            dI = dCS * self.C[t] * sigmoid_derivative(self.I[t])                                   
            dW_i += np.dot(dI, self.X[t].T)
            db_i += dI
            
            # cell gate            
            dC = dCS * self.I[t] * tanh_derivative(self.C[t])
            dW_c += np.dot(dC, self.X[t].T)
            db_c += dC
            
            # forget gate
            dF = dCS * self.CS[t-1] * sigmoid_derivative(self.F[t])
            dW_f += np.dot(dF, self.X[t].T)
            db_f += dF
            
            # input gate
            dZ = np.dot(self.W_f.T, dF) + np.dot(self.W_i.T, dI) + np.dot(self.W_c.T, dC) + np.dot(self.W_o.T, dO)
            
            # error for next time step
            dHS_next = dZ[self.hidden_size:, :]
            dCS_next = self.F[t] * dCS
        
        for d in [dW_f, dW_i, dW_c, dW_o, dW_y, db_f, db_i, db_c, db_o, db_y]:
            np.clip(d, -1, 1, out=d)
        
        self.W_f -= self.learning_rate * dW_f * (-1)
        self.W_i -= self.learning_rate * dW_i * (-1)
        self.W_c -= self.learning_rate * dW_c * (-1)
        self.W_o -= self.learning_rate * dW_o * (-1)
        self.W_y -= self.learning_rate * dW_y * (-1)
        self.b_f -= self.learning_rate * db_f * (-1)
        self.b_i -= self.learning_rate * db_i * (-1)
        self.b_c -= self.learning_rate * db_c * (-1)
        self.b_o -= self.learning_rate * db_o * (-1)
        self.b_y -= self.learning_rate * db_y * (-1)
        
    def train(self, epochs, inputs, labels ):
        for epoch in tqdm(range(epochs)):
            self.reset()            
            predictions = self.forward(inputs)
            errors = []
            for t in range(len(predictions)):
                error = softmax(predictions[t]) - self.one_hot_encoding(self.word_to_idx[labels[t]])
                errors += [error]
            self.backward(errors, inputs)
        
    def test(self, inputs, labels):
        accuracy = 0
        probabilities = self.forward(inputs)
        gt = ''
        output = '나라의 '
        for t in range(len(labels)):
            prediction = self.word_to_idx[np.argmax(softmax(probabilities[t].reshape(-1)))]            
            gt += inputs[t] + ' '
            output += prediction + ' '
            if inputs[t] == prediction:
                accuracy += 1
        print('GT: ', gt)
        print('Output: ', output)
        
    def get_data(self):
       return self.tokens[:-1], self.tokens[1:]
    
    
    
        

In [14]:
MyLSTM = MyLSTM(data, 25, 0.01)
x_train, y_train = MyLSTM.get_data()
MyLSTM.train(1000, x_train, y_train)
MyLSTM.test(x_train, y_train)

  0%|          | 0/1000 [00:00<?, ?it/s]


ValueError: operands could not be broadcast together with shapes (25,1) (44,1) 